In [1]:
# py35

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys, os

In [3]:
import simpy
import random

In [4]:
def get_process(env, name, duration_func):
    
    def p_gen(env):
        while True:
            
            t = duration_func()
            print('[{0}] {1}_start'.format(env.now, name))
            yield env.timeout(t)
            print('[{0}] {1}_stop'.format(env.now, name))
            
    p = env.process(p_gen(env))
    return p
    

In [5]:
env = simpy.Environment()

p1 = get_process(env, 'p1', lambda: 5)
p2 = get_process(env, 'p2', lambda: 5 + random.random())

env.run(until=100)


[0] p1_start
[0] p2_start
[5] p1_stop
[5] p1_start
[5.442309522232732] p2_stop
[5.442309522232732] p2_start
[10] p1_stop
[10] p1_start
[10.958066955648398] p2_stop
[10.958066955648398] p2_start
[15] p1_stop
[15] p1_start
[16.292531660299353] p2_stop
[16.292531660299353] p2_start
[20] p1_stop
[20] p1_start
[21.870869652140975] p2_stop
[21.870869652140975] p2_start
[25] p1_stop
[25] p1_start
[27.683502310660767] p2_stop
[27.683502310660767] p2_start
[30] p1_stop
[30] p1_start
[32.84007706416086] p2_stop
[32.84007706416086] p2_start
[35] p1_stop
[35] p1_start
[38.09371608368415] p2_stop
[38.09371608368415] p2_start
[40] p1_stop
[40] p1_start
[44.054020955059435] p2_stop
[44.054020955059435] p2_start
[45] p1_stop
[45] p1_start
[49.128397721793945] p2_stop
[49.128397721793945] p2_start
[50] p1_stop
[50] p1_start
[54.43337356378124] p2_stop
[54.43337356378124] p2_start
[55] p1_stop
[55] p1_start
[59.70377565518312] p2_stop
[59.70377565518312] p2_start
[60] p1_stop
[60] p1_start
[64.758257879

In [6]:
def parse_transition_func(tf_dict):
    pe = dict()
    for se_pair, new_s in tf_dict.items():
        s, e = se_pair
        if s not in pe:
            pe[s] = [e]
        else:
            pe[s].append(e)

    return pe

def get_random_next_event(fsm):
    ce = fsm.current_events
    n = len(ce)
    if n == 1:
        return ce[0]
    else:
        i = random.randint(0, n-1)
        return ce[i]

class FSM:
    def __init__(self, start, transition_func):
        self.start = start
        self.transition_func = transition_func
        self.possible_events = parse_transition_func(transition_func)
        
        self.current = start
        
    @property
    def current_events(self):
        return self.possible_events[self.current]
    
    def advance(self, event):
        if not event in self.current_events:
            raise Exception()
        
        self.current = self.transition_func[(self.current, event)]
        
    def advance_random(self):
        self.advance(get_random_next_event(self))
        

class SimpleProcess:
    '''
    An independent process with the defined duration function
    '''

    def __init__(self, name, env, duration_func, fsm):
        self.name = name
        self.env = env
        self.duration_func = duration_func
        self.fsm = fsm
        
        self.stats = {'total_time': 0}
        
        self.action = env.process(self.p_gen())
    
    def p_gen(self):
        while True:
            
            next_event = get_random_next_event(self.fsm)
            next_state = self.fsm.transition_func[(self.fsm.current, next_event)]
            
            t = self.duration_func[next_state]()
            self.stats['total_time'] += t
            
            #self.print_event(next_event)
            self.fsm.advance(next_event)
            
            print('now={0:.4f}\t {1}:{2}!->{3} (will take {4})'.format(self.env.now, self.name, next_event, self.fsm.current, t))
            yield env.timeout(t)
            
    def print_event(self, event_name):
        print('[{0}] {1}_{2}'.format(self.env.now, self.name, event_name))
        
class ResourceUser(SimpleProcess):
    def process_cycle(self):
        print('Hello')
        super(ResourceUser, self).process_cycle()

In [7]:
env = simpy.Environment()

simple_fsm = FSM('idle', {('idle', 'start'): 'running', ('running', 'stop'): 'idle'})

p1 = SimpleProcess('p1', env, {'idle': lambda: 1, 'running': lambda: 5}, simple_fsm)
p2 = SimpleProcess('p2', env, {'idle': lambda: 1, 'running': lambda: 5 + random.random()}, simple_fsm)

env.run(until=50)

now=0.0000	 p1:start!->running (will take 5)
now=0.0000	 p2:stop!->idle (will take 1)
now=1.0000	 p2:start!->running (will take 5.214945972898544)
now=5.0000	 p1:stop!->idle (will take 1)
now=6.0000	 p1:start!->running (will take 5)
now=6.2149	 p2:stop!->idle (will take 1)
now=7.2149	 p2:start!->running (will take 5.1010464591823546)
now=11.0000	 p1:stop!->idle (will take 1)
now=12.0000	 p1:start!->running (will take 5)
now=12.3160	 p2:stop!->idle (will take 1)
now=13.3160	 p2:start!->running (will take 5.981298522526966)
now=17.0000	 p1:stop!->idle (will take 1)
now=18.0000	 p1:start!->running (will take 5)
now=19.2973	 p2:stop!->idle (will take 1)
now=20.2973	 p2:start!->running (will take 5.24123836489194)
now=23.0000	 p1:stop!->idle (will take 1)
now=24.0000	 p1:start!->running (will take 5)
now=25.5385	 p2:stop!->idle (will take 1)
now=26.5385	 p2:start!->running (will take 5.167138459264913)
now=29.0000	 p1:stop!->idle (will take 1)
now=30.0000	 p1:start!->running (will take 5)
n

In [8]:
p1.stats, p2.stats

({'total_time': 53}, {'total_time': 51.33842587523609})

In [9]:
env = simpy.Environment()
rsc = simpy.Resource(env, capacity=1)

In [10]:
d1 = {'a': 1, 'b': 2, 'd': 4}
d2 = {'a': 1, 'c': 12, 'e': 31}


common = []
diff1 = []
diff2 = []
for el in d1:
    if el in d2:
        common.append(el)
    else:
        diff1.append(el)

for el in d2:
    if el not in d1:
        diff2.append(el)
        
print(common)
print(diff1)
print(diff2)


['a']
['b', 'd']
['e', 'c']


In [11]:
lst = [0, -10, 5, 6, -4]
def f(i):
    return i+2
def g(i):
    return i>5


for el in lst:
    incr = f(el)
    print(incr, g(incr))

2 False
-8 False
7 True
8 True
-2 False


In [12]:
lst = [1, 3, ['hello', 4, [[13.2, 45, 'world']], 'boo-ya!', 31], 12]
lst

[1, 3, ['hello', 4, [[13.2, 45, 'world']], 'boo-ya!', 31], 12]

In [13]:
def flat_list(lst):

    new_list = []
    for el in lst:
        if not type(el) is list:
            new_list.append(el)
        else:
            unwrapped = flat_list(el)
            new_list = new_list + unwrapped
    
    return new_list
            
flat_list(lst)

[1, 3, 'hello', 4, 13.2, 45, 'world', 'boo-ya!', 31, 12]